In [ ]:
# Cell 1: Imports and Environment Setup
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import openpyxl
import sys
import os
from pathlib import Path

# 현재 작업 경로를 시스템 경로에 추가하여 로컬 모듈 인식 보장
sys.path.append(os.getcwd())


try:
    from Weight_Estimation import WeightEst
    from Conversions import Conversions
    from ambiance import Atmosphere
    from models.stack_functions import cell_model, stack_model, mass_flow_stack
    from models.compressor_performance import compressor_performance_model
    from models.compressor_mass import compressor_mass_model
    from models.humidifier import humidifier_model
    from models.heat_exchanger import heat_exchanger_model
    print("All modules imported successfully.")
except ImportError as e:
    print(f"Error importing modules: {e}")
    print("Ensure 'Weight_Estimation.py', 'Conversions.py', and the 'models' folder are in the current directory.")

# Plot 설정을 위한 magic command
%matplotlib inline


In [ ]:
# Cell 2: FuelCellSystem Class Definition
class FuelCellSystem:
    def __init__(self):
        # 초기화 파라미터들 (현재는 pass 처리되어 있음)
        pass

    def singlecell(self):
        # 단일 셀 모델링 (현재는 pass 처리되어 있음)
        pass

    @staticmethod
    def size_system(power_fc_sys, volt_req, h_cr, mach_cr, oversizing, beta, comp_bool, n_stacks_series):
        """
        Size a fuel cell system.
        """
        figs = []

        # Atmospheric conditions
        atm_cr = Atmosphere(h_cr)
        c_cr = atm_cr.speed_of_sound[0]  # speed of sound at cruise in m
        v_cr = mach_cr * c_cr  # cruise true airspeed in m/s
        p_cr = atm_cr.pressure[0]  # static pressure at cruise altitude in Pa
        p_cr_tot = p_cr * (1 + 0.4 / 2 * mach_cr ** 2) ** (1.4 / 0.4)  # total pressure at cruise in Pa
        t_cr = atm_cr.temperature[0]  # static temperature at cruise altitude in K
        t_cr_tot = t_cr * (1 + 0.4 / 2 * mach_cr ** 2)  # total temperature at cruise in K
        rho_cr = atm_cr.density[0]  # air density at cruise altitude in kg/m3
        mu_cr = atm_cr.dynamic_viscosity[0]  # dynamic viscosity at cruise altitude in Pa s

        # print(f"Reynolds_number: {rho_cr*v_cr*1.8/mu_cr:,.0f}")

        # Other inputs
        cell_temp = 273.15 + 80  # operating temperature inside cell
        mu_f = 0.95  # fuel utilisation

        # Compressor outlet conditions
        if comp_bool:
            pres_cathode_in = beta * p_cr_tot
        else:
            pres_cathode_in = p_cr_tot

        # Cell model
        pres_h = Atmosphere(0).pressure[0]
        volt_cell, power_dens_cell, eta_cell, fig = cell_model(pres_cathode_in, pres_h, cell_temp, oversizing)
        figs.append(fig)

        # Compressor models
        if comp_bool:
            power_req = 0
            power_req_new = power_fc_sys
            while abs(power_req_new - power_req) > 1e-3:
                power_req = power_req_new
                geom_comp, power_comp, rho_humid_in, m_dot_comp = compressor_performance_model(power_req, volt_cell, beta,
                                                                                            p_cr_tot, t_cr_tot, mu_cr)
                power_req_new = power_fc_sys + power_comp
            m_comp = compressor_mass_model(geom_comp, power_comp)
        else:
            m_comp = 0
            power_comp = 0
            power_req_new = power_fc_sys
            m_dot_comp = mass_flow_stack(power_req_new, volt_cell)
            rho_humid_in = rho_cr

        # Remaining BOP models
        m_humid = humidifier_model(m_dot_comp, rho_humid_in)
        q_all, m_hx, dim_hx = heat_exchanger_model(power_req_new, volt_cell, cell_temp, mu_f, v_cr, mach_cr, p_cr_tot, t_cr_tot, rho_cr, mu_cr)

        # Stack model
        m_stacks, dim_stack, res_stack = stack_model(n_stacks_series, volt_req, volt_cell, power_req_new, power_dens_cell)

        # Sum up to find mass of FC system (all masses in kg)
        m_sys = m_stacks + m_comp + m_humid + m_hx

        # Determine FC system efficiency
        eta_fcsys = eta_cell * power_fc_sys / (power_comp + power_fc_sys) * mu_f

        # Hydrogen comsumption
        mdot_h2 = 1.05e-8 * (power_comp + power_fc_sys) / volt_cell

        results = {
            'm_sys': m_sys,
            'm_stacks': m_stacks,
            'm_comp': m_comp,
            'm_humid': m_humid,
            'm_hx': m_hx,
            'eta_fcsys': eta_fcsys,
            'mdot_h2': mdot_h2,
            'power_comp': power_comp,
            'figs': figs,
            'dim_stack': dim_stack,
            'n_stacks_series': n_stacks_series,
            'dim_hx': dim_hx,
            'res_stack': res_stack,
            'q_all': q_all,
            'v_cr': v_cr,
            'm_dot_air': m_dot_comp
            }

        return results

    def othersys(self):
        pass

    def fcstack(self):
        pass


In [ ]:
# Cell 3: FuelCellSizing Class Definition
class FuelCellSizing:
    def __init__(self):
        self.Mission_description = ['ready', 'taxing', 'climb_start', 'climb_end', 'cruise-charger_start', 'cruise-charger_end', 'cruise-FConly_start', 'cruise-FConly_end', 'approach', 'go-around_climb_start', 'go-around_climb_end', 'diversion-cruise-FConly_start', 'diversion-cruise-FConly_end', 'diversion-cruise-FConly_start', 'diversion-cruise-FConly_end', 'approach-final', 'taxing_start', 'taxing_end', 'ready']
        self.Mission_time = [0, 5, 15, 16, 23, 25, 70, 71, 100, 110, 112, 116, 118, 135, 135, 135, 140, 150, 160, 160]
        self.Mission = {k: v for k, v in zip(self.Mission_description, self.Mission_time)}

        self.n_stacks_series = 1
        self.n_stacks_parallel = 4
        self.volt_req = 700  # voltage to be produced by a fuel cell system
        self.h_cr = 3000
        self.mach_cr = 0.35
        self.oversizing = 0.3
        self.beta = 1.05
        self.mdot_cruise = 0.
        self.MTOM = 8600
        self.PFC1 = 200000 # unit(W)
        self.ptotal_a = 0
        self.ptotal_b = 0
        self.ptotal_c = 0
        self.ptotal_d = 0
        self.ptotal_e = 0
        self.ptotal_f = 0
        self.ptotal_g = 0
        self.ptotal_h = 0
        self.mfuel = 0
        self.OEM = 0
        self.W_fus = 0
        self.W_wing = 0
        self.mtank = 0
        self.mem = 0
        self.Pcomp = 0
        self.Pcooling_system = 0
        self.mPMAD = 0
        self.mFC = 0
        self.mcomp = 0
        self.mcoolingsystem = 0
        self.Pheat_rejected = 0
        self.Mfftotal = 0
        self.eta_pt = 0.5
        self.mpt = 0
        self.Mffextra = 0

        #Powertrain parameter
        self.rhofc = 3000
        self.rhopmad = 15000
        self.rhoem = 7000
        self.eta_FC = 0.5
        self.eta_em = 0.9215
        self.eta_storage = 0.4
        self.eta_vol = 0.5
        self.rho_H2 = 70
        self.LHVfuel = 120

        #Pelectric calculation
        self.P_W_climb = 0.1139*1000/9.81
        self.P_W_cruise = 0.0876*1000/9.81
        self.P_W_takeoff = 0.0739*1000/9.81

        #Pcomp
        self.gamma_air = 1.4
        self.Psl = 101325
        self.Palt = 70121
        self.eta_comp = 0.8
        self.lamda_O2 = 1.5
        self.Cp_air = 1005

        #Pcooling
        self.Tair = Atmosphere(self.h_cr).temperature[0]
        self.Tt1 = self.Tair * (1 + 0.4 / 2 * self.mach_cr ** 2)
        self.dT = 60
        self.f_dT = 0.0038 * ((self.Tair/self.dT)**2) + 0.0352 * (self.Tair/self.dT) + 0.1817

        # Calculate temperature rise for compressor
        self.PRcomp = (self.Psl/self.Palt) * 1.05
        self.Tt2 = self.Tt1 * (1 + (1 / self.eta_comp) * ((self.PRcomp)**((self.gamma_air-1)/self.gamma_air)- 1))

        #Fuel cell mass
        self.rhocomp = 2000

        #Hydrogen fuel mass
        self.Rtotal = 500000
        self.hcruise = self.h_cr
        self.hTO = 0
        self.Vv = 8
        self.Vclimb = 110
        self.eta_prop = 0.8
        self.L_D1 = 14.5
        self.L_D2 = 0
        self.g = 9.81
        self.Vtankex = 0.

        #Hydrogen tank geometry
        self.Coversize = 3.0

        #Wing sizing(kg -> lb) *2.205
        self.W_S_imp = Conversions.pa_psf(self, 2830.24, "psf")
        self.AR_wing = 10
        self.t_c = 0.2
        self.nz = 3.*1.5
        self.lc4 = 0
        self.lam = 0.42
        self.c_cruise = Atmosphere(self.h_cr).speed_of_sound[0]
        self.Vcruise = self.mach_cr * self.c_cruise
        self.rho_cruise = Atmosphere(self.h_cr).density[0]
        self.q_cruise = 0.5 * self.rho_cruise * self.Vcruise**2
        self.q_cruise_imp = self.q_cruise * (0.0208855)

        #Fuselage sizing
        self.tanklength = 0
        self.finerat_fus_nose = 0.6
        self.finerat_fus_tail = 0.8
        self.k_w_nose = -0.603291 * self.finerat_fus_nose**2 + 2.17154 * self.finerat_fus_nose + -0.425122
        self.k_w_tail = -0.603291 * self.finerat_fus_tail**2 + 2.17154 * self.finerat_fus_tail + -0.425122
        self.k_c_nose = -1.72626 * self.finerat_fus_nose**3 + 4.43622 * self.finerat_fus_nose**2 + -3.05539 * self.finerat_fus_nose + 1.3414
        self.k_c_nose = -1.72626 * self.finerat_fus_tail**3 + 4.43622 * self.finerat_fus_tail**2 + -3.05539 * self.finerat_fus_tail + 1.3414
        self.Dfus_imp = Conversions.meter_feet(self, 1.85, 'ft')
        self.hfus_imp = self.Dfus_imp
        self.circum_fus_imp = 2*self.k_c_nose * (self.Dfus_imp + self.hfus_imp)
        self.Fnose = 1
        self.Ftail = 2
        self.Lseat = 0.8
        self.Npass = 18
        self.Nseat_abreast = 2
        self.Ldoor = 1
        self.Lnose_imp = self.Dfus_imp * self.Fnose
        self.Ltail_imp = self.Dfus_imp * self.Ftail
        self.Lcabin_imp = ((self.Npass / self.Nseat_abreast) * self.Lseat + self.Ldoor) * 3.281

        self.lHT_imp = 27
        self.dFS_imp = 6.07
        self.lFS_imp = 43.46

        self.S_wing = 0
        self.b_wing = 0

        #Aircraft sizing
        self.OEMmisc = 0
        self.OEMmisc_ = 0

        self.mpayload = 2400
        self.rhobatt = 0.3

        #hybrid parameter
        self.psi_takeoff = 0.25
        self.psi_climb = 0.25
        self.psi_cruise = -0.053
        self.eta_Converter = 0.97
        self.eta_PDU = 0.98
        self.eta_Inverter = 0.97
        self.eta_Motor = 0.95

        self.Pbat_climb = 0.
        self.mair1 = 2.856e-7 * self.lamda_O2 * ((self.ptotal_a - self.Pbat_climb)/ self.eta_FC)
        self.Pcomp1 = 0
        self.Pheat_rejected1 = 0
        self.Pcooling_system1_fc = 0

        self.Pfuelcell_climb = 0.
        self.Pfuelcell_cruise = 0.
        self.Pcomp2 = 0.
        self.Pcooling_system2_fc = 0.

        self.Pbat_cruise_charger = 0
        self.Pfuelcell_cruise_charger = 0

        self.Pcomp3 = 0.
        self.mbatt = 0
        self.Ptotal_climb = 0.

        self.Pfuelcell_takeoff = 0
        self.Pcomp4 = 0
        self.Pbat_takeoff = 0

        self.lndgear = 0
        self.W_HT = 0
        self.W_VT = 0
        self.W_motor = 0
        self.W_flight_control = 0
        self.W_els = 0
        self.W_iae = 0
        self.W_fur = 0
        self.W_hydraulics = 0

    def cal_Ptotal_climb(self, MTOM_trial, PFC1=5000000):
        self.ptotal_a = PFC1
        iter = 1
        while True:
            Pshaft1 = MTOM_trial * 9.81 * self.P_W_climb
            self.Pfuelcell_climb = (1/(self.eta_Inverter*self.eta_Motor*self.eta_PDU*self.eta_prop))*((1-self.psi_climb)/(self.psi_climb+self.eta_Converter*(1-self.psi_climb))) * Pshaft1
            self.Pbat_climb = (1/(self.eta_Inverter*self.eta_Motor*self.eta_PDU*self.eta_prop))*(self.psi_climb/(self.psi_climb+self.eta_Converter*(1-self.psi_climb))) * Pshaft1

            Pelectricnet1 = self.Pfuelcell_climb + self.Pbat_climb

            power_fc_sys_parallel = (self.ptotal_a - self.Pbat_climb)/self.n_stacks_parallel
            if power_fc_sys_parallel <= 0:
                power_fc_sys_parallel = self.ptotal_a
            comp_bool = True

            pemfcsys_sizing_results_climb = FuelCellSystem.size_system(power_fc_sys_parallel, self.volt_req, self.h_cr, self.mach_cr, self.oversizing, self.beta, comp_bool, self.n_stacks_series)

            self.Pcomp1 = pemfcsys_sizing_results_climb['power_comp'] * self.n_stacks_parallel
            self.Pheat_rejected1 = self.n_stacks_parallel*pemfcsys_sizing_results_climb['q_all']/1000

            self.mdot_climb = pemfcsys_sizing_results_climb['mdot_h2'] * self.n_stacks_parallel
            self.Pcooling_system1_fc = (0.371 * self.Pheat_rejected1 + 1.33) * self.f_dT * 1000

            self.ptotal_b = Pelectricnet1 + self.Pcomp1 + self.Pcooling_system1_fc

            if abs(self.ptotal_b - self.ptotal_a) <= 100:
                print(f"------------------------------------------------------")
                print(f"----------------------- iter {iter} -----------------------")
                print(f"Pshaft1: {Pshaft1/1000:,.0f} kW, Pcomp1_fc: {self.Pcomp1/1000:,.0f} kW, Pcooling_system1_fc: {self.Pcooling_system1_fc/1000:,.0f} kW, Pfuelcell_climb: {self.Pfuelcell_climb/1000:,.0f} kW, Pbat_climb: {self.Pbat_climb/1000:,.0f} kW, Ptotal_climb: {self.ptotal_b/1000:,.0f} kW")
                break
            else:
                print(f"cal_Ptotal_climb")
                self.ptotal_a = self.Pfuelcell_climb + self.Pcomp1 + self.Pcooling_system1_fc + self.Pbat_climb
                iter += 1

        return self.ptotal_b, MTOM_trial

    def cal_Ptotal_cruise(self, MTOM_trial, PFC2=5000000):
        self.ptotal_c = PFC2
        while True:
            Pshaft2 = MTOM_trial * 9.81 * self.P_W_cruise
            self.Pfuelcell_cruise = (1/(self.eta_Inverter*self.eta_Motor*self.eta_PDU*self.eta_prop))*((1/self.eta_Converter)) * Pshaft2

            Pelectricnet2 = self.Pfuelcell_cruise

            power_fc_sys_parallel = self.ptotal_c/self.n_stacks_parallel
            comp_bool = True

            pemfcsys_sizing_results_cruise = FuelCellSystem.size_system(power_fc_sys=power_fc_sys_parallel, volt_req=self.volt_req, h_cr=self.h_cr, mach_cr=self.mach_cr, oversizing=self.oversizing, beta=self.beta, comp_bool=comp_bool, n_stacks_series=self.n_stacks_series)

            self.mdot_cruise = pemfcsys_sizing_results_cruise['mdot_h2'] * self.n_stacks_parallel

            self.Pcomp2 = pemfcsys_sizing_results_cruise['power_comp'] * self.n_stacks_parallel
            Pheat_rejected2 = self.n_stacks_parallel*pemfcsys_sizing_results_cruise['q_all']/1000

            self.Pcooling_system2_fc = (0.371 * Pheat_rejected2 + 1.33) * self.f_dT * 1000
            self.ptotal_d = Pelectricnet2 + self.Pcomp2 + self.Pcooling_system2_fc

            if abs(self.ptotal_d - self.ptotal_c) <= 100:
                break
            else:
                print(f"cal_Ptotal_cruise")
                self.ptotal_c = self.Pfuelcell_cruise + self.Pcomp2 + self.Pcooling_system2_fc

        return self.ptotal_d, MTOM_trial


    def cal_Ptotal_cruise_charger(self, MTOM_trial, PFC3=5000000):
        self.ptotal_e = PFC3
        while True:
            Pshaft3 = MTOM_trial * 9.81 * self.P_W_cruise
            self.Pfuelcell_cruise_charger = (1/(self.eta_Inverter*self.eta_Motor*self.eta_PDU*self.eta_prop))*((1-self.psi_cruise)/(self.psi_cruise+self.eta_Converter*(1-self.psi_cruise))) * Pshaft3
            self.Pbat_cruise_charger = (1/(self.eta_Inverter*self.eta_Motor*self.eta_PDU*self.eta_prop))*(self.psi_cruise/(self.psi_cruise+self.eta_Converter*(1-self.psi_cruise))) * Pshaft3

            Pelectricnet3 = self.Pfuelcell_cruise_charger + abs(self.Pbat_cruise_charger)

            comp_bool = True
            power_fc_sys_parallel = self.ptotal_e/self.n_stacks_parallel
            pemfcsys_sizing_results_cruise_charger = FuelCellSystem.size_system(power_fc_sys_parallel, self.volt_req, self.h_cr, self.mach_cr, self.oversizing, self.beta, comp_bool, self.n_stacks_series)

            self.Pcomp3 = pemfcsys_sizing_results_cruise_charger['power_comp'] * self.n_stacks_parallel
            Pheat_rejected3 = self.n_stacks_parallel * pemfcsys_sizing_results_cruise_charger['q_all']/1000
            Pcooling_system3_fc = (0.371 * Pheat_rejected3 + 1.33) * self.f_dT * 1000

            self.ptotal_f = Pelectricnet3 + self.Pcomp3 + Pcooling_system3_fc

            if abs(self.ptotal_f - self.ptotal_e) <= 100:
                break
            else:
                print(f"cal_Ptotal_cruise_charger")
                self.ptotal_e = self.Pfuelcell_cruise_charger + self.Pcomp3 + Pcooling_system3_fc + abs(self.Pbat_cruise_charger)

        return self.ptotal_f, Pcooling_system3_fc, MTOM_trial


    def cal_Ptotal_takeoff(self, MTOM_trial, PFC4=500000):
        self.ptotal_g = PFC4
        while True:
            Pshaft4 = MTOM_trial * 9.81 * self.P_W_takeoff
            self.Pfuelcell_takeoff = (1/(self.eta_Inverter*self.eta_Motor*self.eta_PDU*self.eta_prop))*((1-self.psi_takeoff)/(self.psi_takeoff+self.eta_Converter*(1-self.psi_takeoff))) * Pshaft4
            self.Pbat_takeoff = (1/(self.eta_Inverter*self.eta_Motor*self.eta_PDU*self.eta_prop))*(self.psi_takeoff/(self.psi_takeoff+self.eta_Converter*(1-self.psi_takeoff))) * Pshaft4

            Pelectricnet4 = self.Pfuelcell_takeoff + self.Pbat_takeoff

            power_fc_sys_parallel = (self.ptotal_g - self.Pbat_takeoff)/self.n_stacks_parallel
            if power_fc_sys_parallel <= 0:
                power_fc_sys_parallel = self.ptotal_g
            comp_bool = True

            pemfcsys_sizing_results_takeoff = FuelCellSystem.size_system(power_fc_sys=power_fc_sys_parallel, volt_req=self.volt_req, h_cr=0., mach_cr=0.16, oversizing=self.oversizing, beta=1.05, comp_bool=comp_bool, n_stacks_series=self.n_stacks_series)

            self.Pcomp4 = pemfcsys_sizing_results_takeoff['power_comp'] * self.n_stacks_parallel
            Pheat_rejected4 = self.n_stacks_parallel*pemfcsys_sizing_results_takeoff['q_all']/1000

            Pcooling_system4_fc = (0.371 * Pheat_rejected4 + 1.33) * self.f_dT * 1000
            self.ptotal_h = Pelectricnet4 + self.Pcomp4 + Pcooling_system4_fc

            if abs(self.ptotal_h - self.ptotal_g) <= 100:
                break
            else:
                print(f"cal_Ptotal_takeoff")
                self.ptotal_g = self.Pfuelcell_takeoff + self.Pcomp4 + Pcooling_system4_fc + self.Pbat_takeoff

        return self.ptotal_h, MTOM_trial


    def cal_MTOM(self, MTOM_trial):
        MTOM = MTOM_trial

        power_fc_sys_parallel = self.Pfuelcell_climb/self.n_stacks_parallel
        comp_bool = True
        pemfcsys_sizing_results = FuelCellSystem.size_system(power_fc_sys_parallel, self.volt_req, self.h_cr, self.mach_cr, self.oversizing, self.beta, comp_bool, self.n_stacks_series)

        print(f"Stack(1/{self.n_stacks_parallel}): {pemfcsys_sizing_results['m_stacks']:,.0f} kg, Compressor: {pemfcsys_sizing_results['m_comp']:,.0f} kg, Humidifier: {pemfcsys_sizing_results['m_humid']:,.0f} kg, HX: {pemfcsys_sizing_results['m_hx']:,.0f} kg")
        print(f"Power density of Nacelle System: {power_fc_sys_parallel/1000/pemfcsys_sizing_results['m_sys']:,.3f} kW/kg")
        print(f"dim_hx: dX = {pemfcsys_sizing_results['dim_hx'][0]:,.3f} m, dY = {pemfcsys_sizing_results['dim_hx'][1]:,.3f} m, dZ = {pemfcsys_sizing_results['dim_hx'][2]:,.3f} m")
        print(f"dim_stack: dX = {pemfcsys_sizing_results['dim_stack'][2]:,.3f} m, dY = {pemfcsys_sizing_results['dim_stack'][0]:,.3f} m, dZ = {pemfcsys_sizing_results['dim_stack'][1]:,.3f} m")

        # Calculate powertrain component masses
        self.mFC = self.n_stacks_parallel * (pemfcsys_sizing_results['m_stacks'] + pemfcsys_sizing_results['m_humid'] + pemfcsys_sizing_results['m_comp'] + pemfcsys_sizing_results['m_hx'])
        mbatt_old = (self.Pbat_climb * 0.234) / (self.rhobatt * 1000)
        self.mbatt = mbatt_old/(1 - 0.25)
        self.mcomp = (self.n_stacks_parallel)*pemfcsys_sizing_results['m_comp']

        self.mcoolingsystem = (self.n_stacks_parallel)*pemfcsys_sizing_results['m_hx']
        print(f"mcoolingsystem: {self.mcoolingsystem:,.0f} kg")

        PPDU = (self.Pfuelcell_climb + self.Pheat_rejected1 + self.Pcooling_system1_fc) * self.eta_Converter + self.Pbat_climb
        Pem = (self.Pfuelcell_climb * self.eta_Converter + self.Pbat_climb) * self.eta_PDU * self.eta_Inverter

        self.mem = Pem / self.rhoem
        self.mPMAD = PPDU / self.rhopmad

        self.mpt = (self.mFC + self.mPMAD + self.mem)

        # Calculate hydrogen fuel mass
        tclimb = (self.hcruise - self.hTO) / self.Vv
        self.Vclimb = math.sqrt(self.Vv**2 + (self.Vcruise)**2)
        Rclimb = self.Vclimb * tclimb
        Rdescent = Rclimb
        Rcruise = self.Rtotal - Rclimb - Rdescent

        self.Ptotal_climb = self.Pfuelcell_climb + self.Pbat_climb + self.Pcomp1 + self.Pcooling_system1_fc
        Pfuelcell_engine = self.Ptotal_climb * 0.05
        Pfuelcell_taxing = self.Ptotal_climb * 0.1

        self.mfuel = self.mdot_cruise*(Rcruise + Rdescent)/pemfcsys_sizing_results['v_cr'] + self.mdot_climb*(Rclimb)/self.Vclimb
        print(f"mfuel: {self.mfuel:,.0f} kg")

        #Hydrogen tank sizing
        VH2_tank = self.mfuel * self.Coversize / self.rho_H2 / self.eta_vol
        self.Vtankex = VH2_tank
        Ltank = self.Vtankex / (math.pi * ((self.Dfus_imp / (2 * 3.281))**2))
        self.tanklength = Ltank

        self.mtank = (self.mfuel * self.Coversize / self.eta_storage) - (self.mfuel * self.Coversize)

        #Calculate Wing sizing
        MTOM_imp = Conversions.kg_pound(self, MTOM, "pound")
        self.S_wing_imp = MTOM_imp / self.W_S_imp
        self.b_wing_imp = math.sqrt(self.AR_wing * self.S_wing_imp)

        print(f"b_wing: {Conversions.meter_feet(self, self.b_wing_imp, 'meter'):,.2f} m")

        #Calculate fuselage sizing
        print(f"Ltank: {Ltank:,.2f} m")
        self.Ltank_imp = Ltank * 3.281
        self.Lfus_imp = self.Lnose_imp + self.Ltail_imp + self.Lcabin_imp + self.Ltank_imp

        self.Swet_fus_imp = self.circum_fus_imp *((self.Lcabin_imp + self.Ltank_imp) + self.k_w_nose*self.Lnose_imp + self.k_w_tail*self.Ltail_imp)

        print(f"Lnose: {Conversions.meter_feet(self, self.Lnose_imp, 'meter'):,.2f} m, Lcabin: {Conversions.meter_feet(self, self.Lcabin_imp, 'meter'):,.2f} m, Ltail: {Conversions.meter_feet(self, self.Ltail_imp, 'meter'):,.2f} m")
        print(f"Lfus: {Conversions.meter_feet(self, self.Lfus_imp, 'meter'):,.2f} m")

        WEst = WeightEst(W=MTOM, W_fw=0., S=Conversions.m2_ft2(self, self.S_wing_imp, 'm2'),
                         b=Conversions.meter_feet(self, self.b_wing_imp, 'meter'), rho_a_cruise=self.rho_cruise,
                         v_cruise=self.Vcruise, t_r_HT=0.09, S_HT=4.5404, S_VT=4.4004, t_r_VT=0.09*1.361,
                         L_HT_act=(Conversions.meter_feet(self, self.lHT_imp, "meter")), b_HT=4.765, b_VT=2.907,
                         FL=(Conversions.meter_feet(self, self.Lfus_imp, "meter")),
                         Wf_mm=(Conversions.meter_feet(self, self.Dfus_imp, "meter")*1000),
                         hf_mm=(Conversions.meter_feet(self, self.hfus_imp, "meter")*1000),
                         W_press=0, l_n_mm=(Conversions.meter_feet(self, self.Lnose_imp, "meter")*1000),
                         Croot=2.430, tc_r=self.t_c, n_ult=self.nz,
                         Swet_fus=(Conversions.m2_ft2(self, self.Swet_fus_imp, "m2")))
        [self.W_wing_imp, self.W_wing, self.W_HT_imp, self.W_HT, self.W_VT_imp, self.W_VT, self.W_fus_imp, self.W_fus,
         self.W_lndgearmain_imp, self.W_lndgearmain, self.W_lndgearnose_imp, self.W_lndgearnose] =\
              WEst.raymermethod(A=self.AR_wing, theta_c4=(math.radians(self.lc4)), lamda=self.lam, tc=self.t_c,
                                theta_c4_HT=(math.radians(2.671)), lamda_HT=0.4, theta_c4_VT=(math.radians(19.53)),
                                lamda_VT=0.4, LD=self.L_D1, Kmp=1.0, W_l=Conversions.kg_pound(self, 8000, 'pound'), N_l=3.0*1.5, L_m=Conversions.meter_inch(self, 0.9, "inch"), N_mw=1, N_mss=1, V_stall=Conversions.km1h_kn(self, 58.32, 'kn'), Knp=1.0, L_n=Conversions.meter_inch(self, 0.90, "inch"), N_nw=1, K_uht=1., F_w=Conversions.meter_feet(self, 1.11, 'ft'), L_t=Conversions.meter_feet(self, 10.74, 'ft'), K_y=Conversions.meter_feet(self, 3.22, 'ft'), A_ht=5, S_e=Conversions.m2_ft2(self, 0.73, 'ft2'), H_t__H_v=1., K_z=Conversions.meter_feet(self, 10.7, 'ft'), t_c=0.09)
        [self.W_motor, self.W_flight_control, self.W_els, self.W_iae, self.W_fur, self.W_hydraulics] =\
        WEst.miscweightest(pow_max=380, N_f=6, N_m=2, S_cs=Conversions.m2_ft2(self, 8.36, "ft2"), Iyaw=4.49*10**6, R_kva=50, L_a=Conversions.meter_feet(self, 20, "ft"), N_gen=1, N_e=4, W_TO=Conversions.kg_pound(self, MTOM, "pound"), N_c=2, W_c=Conversions.kg_pound(self, 10*19, "pound"), S_f=self.Swet_fus_imp*23, N_pil=2)

        #Calculate aircraft sizing
        self.OEMmisc = self.OEMmisc_ + self.W_HT + self.W_VT + self.W_lndgearmain + self.W_lndgearnose + self.W_flight_control + self.W_els + self.W_iae  + self.W_hydraulics + self.W_fur
        self.OEM = self.OEMmisc + self.mpt + self.mtank + self.W_wing + self.W_fus + self.mbatt
        MTOM = self.OEM + self.mfuel + self.mpayload
        print(f"MTOM: {MTOM:,.0f} kg")

        return MTOM, Pfuelcell_engine, Pfuelcell_taxing, pemfcsys_sizing_results, power_fc_sys_parallel


In [ ]:
# Cell 4: Parameter Initialization and Main Execution
PFC = 200000
ptotal_cruise = PFC
ptotal_cruise_charger = PFC
ptotal_takeoff = PFC
Ptotal_climb = PFC
mtom_a = 8000
# MTOM_t = 7000

# 인스턴스 생성
FCS = FuelCellSizing()
FCsys = FuelCellSystem()
FCsys.singlecell()

iter2 = 1
while True:
    print(f"======================================================")
    print(f"======================= ITER {iter2} =======================")

    # 각 비행 단계별 파워 계산
    [ptotal_cruise, _] = FCS.cal_Ptotal_cruise(mtom_a, ptotal_cruise)
    [ptotal_cruise_charger, Pcooling_system3_fc, _] = FCS.cal_Ptotal_cruise_charger(mtom_a, ptotal_cruise_charger)
    [ptotal_takeoff, _] = FCS.cal_Ptotal_takeoff(mtom_a, ptotal_takeoff)
    [Ptotal_climb, _] = FCS.cal_Ptotal_climb(mtom_a, Ptotal_climb)

    # MTOM 계산
    [MTOM_t, Pfuelcell_engine, Pfuelcell_taxing, pemfcsys_sizing_results, power_fc_sys_parallel] = FCS.cal_MTOM(mtom_a)

    print(f"ptotal_climb: {FCS.Ptotal_climb/1000:,.0f} kW, mtom_a: {MTOM_t:,.0f} kg")
    print(f"------------------------------------------------------")
    pnet = Ptotal_climb * FCS.eta_PDU * FCS.eta_em

    # 수렴 조건 검사 (오차 1kg 이하)
    if abs(mtom_a-MTOM_t) <= 1:
        print(f"\n\n")
        print(f"=============================================================")
        print(f"========================= CONVERGED =========================")
        print(f"=============================================================")

        # 최종 결과 출력
        print(f"S_wing: {((Conversions.meter_feet(FCS, FCS.b_wing_imp, 'meter'))**2)/FCS.AR_wing:,.2f} m^2")
        print(f"b_wing: {Conversions.meter_feet(FCS, FCS.b_wing_imp, 'meter'):,.2f} m")
        print(f"Lfus: {Conversions.meter_feet(FCS, FCS.Lfus_imp, 'meter'):,.2f} m")
        print(f"Lnose: {Conversions.meter_feet(FCS, FCS.Lnose_imp, 'meter'):,.2f} m, Lcabin: {Conversions.meter_feet(FCS, FCS.Lcabin_imp, 'meter'):,.2f} m, Ltail: {Conversions.meter_feet(FCS, FCS.Ltail_imp, 'meter'):,.2f} m")
        print(f"Ltank: {FCS.tanklength:,.2f} m")

        print(f"Ptotal_climb: {FCS.Ptotal_climb/1000:,.0f} kW")
        print(f"Ptotal_cruise: {ptotal_cruise/1000:,.0f} kW")
        print(f"Ptotal_takeoff: {ptotal_takeoff/1000:,.0f} kW")
        print(f"Pelectricnet: {Ptotal_climb/1000:,.0f} kW")
        print(f"Pcomp: {FCS.Pcomp1/1000:,.0f} kW")
        print(f"Pcoolingsystem: {FCS.Pcooling_system1_fc/1000:,.0f} kW")
        print(f"Pnet: {pnet/1000:,.0f} kW")
        print(f"eta_pt: {FCS.eta_pt:,.4f}")

        print("\nPer Nacelle")
        print(f"Stack(1/{FCS.n_stacks_parallel}): {pemfcsys_sizing_results['m_stacks']:,.0f} kg, Compressor: {pemfcsys_sizing_results['m_comp']:,.0f} kg, Humidifier: {pemfcsys_sizing_results['m_humid']:,.0f} kg, HX: {pemfcsys_sizing_results['m_hx']:,.0f} kg")
        print(f"Power density of Nacelle System: {power_fc_sys_parallel/1000/pemfcsys_sizing_results['m_sys']:,.3f} kW/kg")

        print("\nALL Nacelles")
        print(f"FCS(FC+Humidifier+Comp+Hx): {FCS.mFC:,.0f} kg")
        print(f"mPMAD: {FCS.mPMAD:,.0f} kg")
        print(f"Electric Motors: {FCS.mem:,.0f} kg")

        print(f"\n-----------------------")
        print(f"Powertrain(FCS+PMAD+Motors): {FCS.mpt:,.0f} kg")
        print(f"mtank: {FCS.mtank:,.0f} kg")
        print(f"W_wing: {FCS.W_wing:,.0f} kg")
        print(f"OEM: {FCS.OEM:,.0f} kg")
        print(f"mfuel: {FCS.mfuel:.1f} kg")
        print(f"mbatt: {FCS.mbatt:.1f} kg")
        print(f"mdot_H2: {FCS.mdot_cruise*1000:.1f} g/s")
        print(f"-----------------------")
        print(f"MTOM: {MTOM_t:,.0f} kg")
        print(f"\nVtankex: {FCS.Vtankex:,.1f} m^3")
        print(f"========================== END ==============================")
        break
    else:
        mtom_a = MTOM_t
        iter2 += 1
        print(f"======================================================")
        print(f"\n")


In [ ]:
# Cell 5: Visualization and Export
# Hybrid power calculation for plotting
pfc_ready =  Pfuelcell_engine
pfc_taxing = Pfuelcell_taxing
pfc_climb = Ptotal_climb - FCS.Pbat_climb
pbat_climb = FCS.Pbat_climb
pfc_cruise1 = ptotal_cruise_charger
pfc_cruise2 = ptotal_cruise
pfc_takeoff = ptotal_takeoff - FCS.Pbat_takeoff
pbat_takeoff = FCS.Pbat_takeoff
pbat_charge = FCS.Pbat_cruise_charger

# PEMFC Performance Figure
figs = pemfcsys_sizing_results['figs']
# figs[-1].show() # Jupyter에서는 아래 코드로 이미지 저장 및 표시

# 이미지 저장 폴더 생성 (없을 경우)
save_dir = Path.cwd() / "figs"
save_dir.mkdir(parents=True, exist_ok=True)
save_path = save_dir / "pemfc_fig.png"
# figs[-1].write_image(str(save_path)) # Plotly 설치 및 설정 필요.
print(f"PEMFC figure saved to: {save_path}")

# Power Mission Profile Plot
print(f"\nPfuel_ready: {pfc_ready/1000:,.0f} kW, Pfuel_taxing: {pfc_taxing/1000:,.0f} kW")

Power_fc = [pfc_ready, pfc_taxing, pfc_takeoff, pfc_climb, pfc_climb, pfc_cruise1, pfc_cruise1, pfc_cruise2, pfc_cruise2, pfc_taxing, pfc_climb, pfc_climb, pfc_cruise2, pfc_cruise2, pfc_cruise2, pfc_cruise2, pfc_taxing, pfc_taxing, pfc_taxing, pfc_ready]
Power_bat = [0, 0, pbat_takeoff, pbat_climb, pbat_climb, pbat_charge, pbat_charge, 0, 0, 0, pbat_climb, pbat_climb, 0, 0, 0, 0, 0, 0, 0, 0]

x = FCS.Mission_time
y1 = np.array(Power_bat)/1000
y2 = np.array(Power_fc)/1000
y3 = y1 + y2

plt.figure(figsize=(10, 6))
plt.plot(x, y1, linestyle='solid', color='gray', label='Battery')
plt.plot(x, y2, linestyle='dashed', color='orange', label='Fuel Cell')
plt.plot(x, y3, linestyle='solid', color='blue', label='Total')

plt.xlabel('Time(min)')
plt.ylabel('Power(kW)')
plt.axis([0, 180, -200, 2500])
plt.title('Power Mission Profile')
plt.legend(loc='upper right')
plt.grid(True)

# 그래프 파일 저장
profile_img_name = "Power_Mission_Profile.png"
plt.savefig(profile_img_name, dpi=400)
plt.show()
print(f"Mission profile saved to: {profile_img_name}")

# Excel Export
ReqPow = np.array([y3, y2, y1])
ReqPow = ReqPow.T
df = pd.DataFrame(ReqPow, columns=["ReqPow_AC", "ReqPow_FC", "ReqPow_Batt"])
# df = df.T # Transpose needed? Check logic. Original code had df.T but column names suggest otherwise.
# Original logic creates 3 rows then transposes to columns.
# But providing columns=['...'] to DataFrame constructor with (N,3) array works directly.
# Let's stick to original flow logic:
df_export = pd.DataFrame(ReqPow, columns=["ReqPow_AC", "ReqPow_FC", "ReqPow_Batt"])
df_export.to_excel("ReqPowDATA.xlsx", index=True)
print("Data exported to ReqPowDATA.xlsx")
